### Load an RDF graph

In this example, we will use the `ipyradiant` `FileManager`.

In [ ]:
from ipyradiant import FileManager, PathLoader

lw = FileManager(loader=PathLoader(path="data"))
# here we hard set what we want the file to be, but ideally a user can choose a file to work with.
lw.loader.file_picker.value = lw.loader.file_picker.options["starwars.ttl"]
lw

### GraphExplorer
TODO

In [ ]:
from ipyradiant.visualization.explore import GraphExplorer

In [ ]:
ge = GraphExplorer()
ge.rdf_graph = lw.graph
ge